In [1]:
import git2repo
import pygit2
import re
import pandas as pd
from datetime import datetime
import re, unicodedata
import nltk.corpus
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer
from pygit2 import GIT_SORT_TOPOLOGICAL, GIT_SORT_REVERSE
repo_url = 'git://github.com/rspec/rspec-rails.git'
repo_name = 'rspec-rails'

In [2]:
repo_obj = git2repo.git2repo(repo_url,repo_name)
repo = repo_obj.clone_repo()

In [4]:
list(repo_obj.get_branches())

['master',
 'origin/2-0-stable',
 'origin/2-11-maintenance',
 'origin/2-12-maintenance',
 'origin/2-13-maintenance',
 'origin/2-14-maintenance',
 'origin/2-2-maintenance',
 'origin/2-3-maintenance',
 'origin/2-9-maintenance',
 'origin/2-99-maintenance',
 'origin/3-0-maintenance',
 'origin/3-1-maintenance',
 'origin/3-2-maintenance',
 'origin/3-4-maintenance',
 'origin/3-5-maintenance',
 'origin/3-6-maintenance',
 'origin/3-7-maintenance',
 'origin/3-8-maintenance',
 'origin/active-support-test-case-assertion',
 'origin/assertion-delegate',
 'origin/change_git_to_https',
 'origin/cherry-pick-1951',
 'origin/contributing-updates',
 'origin/fix-no-method-error-render_views',
 'origin/master',
 'origin/move-have-rendered-to-public-api',
 'origin/patch-mocks-json',
 'origin/rails-5-support',
 'origin/rails4-minitest-lifecycle',
 'origin/records-in-rails-4',
 'origin/remove-base-matcher-dependencies',
 'origin/revert-1730-support-for-argument-matchers',
 'origin/rfc-versioning-strategy',
 'o

In [3]:
commits = repo_obj.get_commit_objects()
commit_df = pd.DataFrame(commits, columns = ['commit_object'])
commit_df['isBuggy'] = pd.Series([0]*commit_df.shape[0])

In [4]:
committed_files = repo_obj.get_committed_files()

In [5]:
diffs = repo_obj.get_diffs(commits)

TypeError: <_pygit2.Commit object at 0x00000204846075B0>

In [6]:
def isBuggyCommit(commit):
    res=re.search(r'\b{bug|defect|fix|patch|#}\b',stemming(commit.message),re.IGNORECASE)
    if res is not None:
        return True

def stemming(x):
    port_stem = PorterStemmer()
    words = []
    for word in x.split(' '):
        words.append(port_stem.stem(word))
    return ' '.join(words)

In [7]:
for i in range(commit_df.shape[0]):
    result = isBuggyCommit(commit_df.loc[i,'commit_object'])
    if result:
        commit_df.loc[i,'isBuggy'] = 1
    else:
        commit_df.loc[i,'isBuggy'] = 0

In [8]:
buggy_commit_df = commit_df[commit_df['isBuggy'] == 1]
buggy_diffs = repo_obj.get_diffs(buggy_commit_df['commit_object'].values.tolist())
bug_creator = []
for value in buggy_diffs:
    _diff_files = buggy_diffs[value]['files']
    repo.head.set_target(buggy_diffs[value]['object'].parent_ids[0])
    for _value in _diff_files:
        try:
            file_path = _diff_files[_value]['file_path']
            blame = repo_obj.get_blame(file_path)
            for _line in _diff_files[_value]['old_lines']:
                if _line != -1:
                    ref = blame.for_line(_line)
                    bug_creator.append([ref.final_committer.name, ref.orig_commit_id, 1])
        except:
            continue

In [11]:
pygit2.discover_repository(repo_obj.repo_path)
repo = pygit2.Repository(pygit2.discover_repository(repo_obj.repo_path))

In [92]:
repo_obj.repo_remove()

In [43]:
bug_creator_df = pd.DataFrame(bug_creator, columns = ['committer','commit','ob'])
bug_creator_df = bug_creator_df.drop_duplicates()
df = bug_creator_df.groupby( ['committer']).count()
defect_count = []
for key,value in df.iterrows():
    user = key
    count = value.values.tolist()[0]
    defect_count.append([user,count])

In [87]:
committer_count = []
for i in range(commit_df.shape[0]):
    commit_id = commit_df.loc[i,'commit_object'].id
    user = commit_df.loc[i,'commit_object'].committer
    committer_count.append([user.name, commit_id, 1])
committer_count_df = pd.DataFrame(committer_count, columns = ['committer', 'commit_id', 'ob'])
committer_count_df = committer_count_df.drop_duplicates()
df = committer_count_df.groupby( ['committer']).count()
commit_count = []
for key,value in df.iterrows():
    user = key
    count = value.values.tolist()[0]
    commit_count.append([user,count])

In [90]:
commit_count

[['Aaron Blew', 1],
 ['Aaron Kromer', 290],
 ['Abdelkader Boudih', 2],
 ['AbuSabah', 1],
 ['Ahmed AbouElhamayed', 2],
 ['Al Snow', 1],
 ['Alan Shields', 2],
 ['Alex Egan', 4],
 ['Alex Rothenberg', 8],
 ['Alex Yaremyuk', 1],
 ['Andre Arko & Doc Ritezel', 1],
 ['Andrei Maxim', 1],
 ['Andrew White', 2],
 ['Andrew Wilcox', 1],
 ['Andrey Voronkov', 1],
 ['André Arko', 1],
 ['Andy Lindeman', 318],
 ['Andy Morris', 1],
 ['Andy Waite', 1],
 ['Anthony Dmitriyev', 1],
 ['Anton Davydov', 1],
 ['Anton Rieder', 1],
 ['Ara Hacopian', 1],
 ['Ari Pollak', 1],
 ['Artur Roszczyk', 1],
 ['Attila Györffy', 2],
 ['Ben Hamill', 3],
 ['Ben Morris', 1],
 ['Ben Moss', 1],
 ['Benjamin Quorning', 9],
 ['Benoit Tigeot', 14],
 ['Billy', 5],
 ['Bradley Schaefer', 26],
 ['Brent J. Nordquist', 1],
 ['Brian Cardarella', 1],
 ['Brian Fontenot', 2],
 ['Brian John', 1],
 ['Brian Kane', 2],
 ['Brian Kephart', 1],
 ['Bryan Helmkamp', 1],
 ['Bryan Ricker', 1],
 ['Carlhuda', 3],
 ['Chris Thorn', 1],
 ['Chris Your', 1],
 ['C

In [91]:
defect_count

[['Aaron Blew', 1],
 ['Aaron Kromer', 108],
 ['Abdelkader Boudih', 2],
 ['Al Snow', 1],
 ['Alan Shields', 2],
 ['Alex Egan', 1],
 ['Alex Rothenberg', 4],
 ['Alyssa Ross', 1],
 ['Andre Arko', 1],
 ['Andreas Neuhaus', 2],
 ['Andrew White', 3],
 ['Andrew Wilcox', 1],
 ['André Arko', 1],
 ['Andy Lindeman', 114],
 ['Andy Morris', 1],
 ['Anthony Dmitriyev', 1],
 ['Anton Davydov', 1],
 ['Ara Hacopian', 1],
 ['Benjamin Quorning', 5],
 ['Benoit Tigeot', 1],
 ['Billy', 4],
 ['Bradley Schaefer', 5],
 ['Brent J. Nordquist', 1],
 ['Brian Fontenot', 3],
 ['Bryan Ricker', 1],
 ['Carlhuda', 1],
 ['Christof', 1],
 ['Darryl Pogue', 1],
 ['DarthSim', 2],
 ['Dave Woodall', 1],
 ['David Chelimsky', 359],
 ['David Chelimsky and Josh Graham', 1],
 ['David Rodríguez', 3],
 ['Diego Carrion', 1],
 ['Diego Plentz', 3],
 ['Diogo Benicá', 1],
 ['Douglas Lovell', 1],
 ['Eliot Sykes', 1],
 ['Erik Michaels-Ober', 2],
 ['Franklin Webber', 1],
 ['Gabe Martin-Dempesy', 2],
 ['Giovanni Cappellotto', 2],
 ['Griffin Smith'